In [2]:
import csv
import os

import matplotlib.pyplot as plt
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from PIL import Image
from mediapipe.framework.formats.landmark_pb2 import NormalizedLandmarkList

In [3]:
def get_facemesh_coords(
    landmark_list: NormalizedLandmarkList, img: np.ndarray, normalize: bool = True
) -> np.ndarray:
    """Extract FaceMesh landmark coordinates into 468x3 NumPy array."""
    h, w = img.shape[:2]  # grab width and height from image
    xyz = [(lm.x, lm.y, lm.z) for lm in landmark_list.landmark]
    if normalize:
        return np.array(xyz).astype(float)
    return np.multiply(xyz, [w, h, w]).astype(int)

In [4]:
data_dir = Path("../data/photographed_data")
visualize_dir = Path("../data/mediapipe_visualize")
meshrender_dir = Path("../data/mediapipe_render")
if not visualize_dir.exists(): visualize_dir.mkdir()
if not meshrender_dir.exists(): meshrender_dir.mkdir()

is_save_image = False
is_render_mesh = False
data_table = pd.read_csv("../data/photographed_data/processed_data_table.csv")

left_iris_index = {474,475, 476, 477}
right_iris_index = {469, 470, 471, 472}
left_eye_index = {362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 }
right_eye_index = {33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 }
facemesh_index = {_ for _ in range(478)}-left_iris_index-right_iris_index-left_eye_index-right_eye_index

iris_idx = sorted(list(left_iris_index | right_iris_index))
eye_idx = sorted(list(left_eye_index | right_eye_index))
face_idx = sorted(list(facemesh_index))

iris_columns = [f"iris{e}_{c}" for e in iris_idx for c in ["x", "y", "z"]]
eye_columns = [f"eye{e}_{c}" for e in eye_idx for c in ["x", "y", "z"]]
face_columns = [f"face{e}_{c}" for e in face_idx for c in ["x", "y", "z"]]

print(len(iris_columns) + len(eye_columns) + len(face_columns))

1434


In [5]:

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh
with mp_face_mesh.FaceMesh(
        static_image_mode=False,
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5,
    ) as face_mesh:
    face_mesh_table = np.zeros(shape=(len(data_table), 478*3))
    relative_img_path_list = []
    for i, relative_img_path in enumerate(tqdm(data_table["PATH"].values)):
        mesh_point_cnt = 0
        relative_img_path_list.append(relative_img_path)
        # image = np.array(Image.open("../datasets/photographed_data/C/WIN_20221122_16_15_25_Pro_00000060.jpg"))
        img_path = data_dir.joinpath(relative_img_path)
        image = np.array(Image.open(img_path))
        render = np.zeros(image.shape, dtype=np.uint8)
        results = face_mesh.process(image)
        if results.multi_face_landmarks is not None:
            face_landmarks = results.multi_face_landmarks[0]
            coords = get_facemesh_coords(face_landmarks, image, normalize=False)
            for e in sorted(list(left_iris_index | right_iris_index)):
                landmark = face_landmarks.landmark[e]
                # print(mesh_point_cnt*3 + 0)
                face_mesh_table[i, mesh_point_cnt*3 + 0] = landmark.x
                face_mesh_table[i, mesh_point_cnt*3 + 1] = landmark.y
                face_mesh_table[i, mesh_point_cnt*3 + 2] = landmark.z

                if is_save_image:
                    image = cv2.circle(
                                image, (coords[e, 0], coords[e, 1]), 3, (255, 0, 0), thickness=-1
                            )
                if is_render_mesh:
                    render = cv2.circle(
                                render, (coords[e, 0], coords[e, 1]), 3, (255, 0, 255), thickness=-1
                            )
                mesh_point_cnt += 1
            for e in sorted(list(left_eye_index | right_eye_index)):
                landmark = face_landmarks.landmark[e]
                face_mesh_table[i, mesh_point_cnt*3 + 0] = landmark.x
                face_mesh_table[i, mesh_point_cnt*3 + 1] = landmark.y
                face_mesh_table[i, mesh_point_cnt*3 + 2] = landmark.z

                if is_save_image:
                    image = cv2.circle(
                        image, (coords[e, 0], coords[e, 1]), 2, (0, 255, 0)
                    )
                if is_render_mesh:
                    render = cv2.circle(
                        render, (coords[e, 0], coords[e, 1]), 2, (0, 255, 0)
                    )
                mesh_point_cnt += 1
            for e in sorted(list(facemesh_index)):
                landmark = face_landmarks.landmark[e]
                face_mesh_table[i, mesh_point_cnt*3 + 0] = landmark.x
                face_mesh_table[i, mesh_point_cnt*3 + 1] = landmark.y
                face_mesh_table[i, mesh_point_cnt*3 + 2] = landmark.z
                
                if is_save_image:
                    image = cv2.circle(
                        image, (coords[e, 0], coords[e, 1]), 1, (0, 0, 100)
                    )
                if is_render_mesh:
                    render = cv2.circle(
                        render, (coords[e, 0], coords[e, 1]), 1, (0, 179, 239)
                    )
                mesh_point_cnt += 1
        else:
            # print("Detection Failure")
            pass
        if not visualize_dir.joinpath(relative_img_path).parent.exists():
            visualize_dir.joinpath(relative_img_path).parent.mkdir()
        if not meshrender_dir.joinpath(relative_img_path).parent.exists():
            meshrender_dir.joinpath(relative_img_path).parent.mkdir()
        if is_save_image: Image.fromarray(image).save(visualize_dir.joinpath(relative_img_path))
        if is_render_mesh: Image.fromarray(render).save(meshrender_dir.joinpath(relative_img_path))
        # break
    

    face_mesh_df = pd.DataFrame(face_mesh_table, index=relative_img_path_list, columns=iris_columns+eye_columns+face_columns)
    face_mesh_df.to_csv(visualize_dir.joinpath("facemesh.csv"))

# plt.imshow(image)
# plt.show()
# image.shape

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection Failure
Detection 

In [6]:
face_mesh_df.reset_index().rename(columns={"index": "PATH"})

,PATH,iris469_x,iris469_y,iris469_z,iris470_x,iris470_y,iris470_z,iris471_x,iris471_y,iris471_z,...,face465_z,face467_x,face467_y,face467_z,face468_x,face468_y,face468_z,face473_x,face473_y,face473_z
0,A/WIN_20221122_15_59_43_Pro_00002521.jpg,0.050921,0.535985,0.020243,0.041640,0.524391,0.020243,0.034509,0.538844,0.020243,...,-0.039480,0.187353,0.491354,-0.062566,0.042635,0.537606,0.020243,0.136263,0.517795,-0.051283
1,A/WIN_20221122_15_59_43_Pro_00002522.jpg,0.129665,0.522859,0.038498,0.117217,0.512172,0.038498,0.109517,0.530225,0.038498,...,-0.014076,0.252512,0.468210,-0.029648,0.119409,0.526774,0.038498,0.213271,0.505276,-0.022296
2,A/WIN_20221122_15_59_43_Pro_00002523.jpg,0.151061,0.535511,0.053037,0.140435,0.528136,0.053037,0.136560,0.544524,0.053037,...,-0.009218,0.263407,0.437156,-0.030716,0.143825,0.540142,0.053037,0.235843,0.473923,-0.020631
3,A/WIN_20221122_15_59_43_Pro_00002524.jpg,0.223679,0.498827,0.056931,0.213074,0.493173,0.056931,0.205851,0.510333,0.056931,...,0.003325,0.334562,0.413146,-0.004158,0.214616,0.504681,0.056931,0.308811,0.437239,0.001437
4,A/WIN_20221122_15_59_43_Pro_00002525.jpg,0.267782,0.469291,0.055267,0.253962,0.460605,0.055267,0.243765,0.479047,0.055267,...,-0.001398,0.374796,0.405864,-0.011049,0.255959,0.474164,0.055267,0.337217,0.431811,-0.004910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7329,H/WIN_20221122_15_59_43_Pro_00001253.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7330,H/WIN_20221122_15_59_43_Pro_00001254.jpg,0.499172,0.622574,-0.023337,0.487309,0.608572,-0.023337,0.473995,0.620989,-0.023337,...,0.002184,0.592833,0.623461,0.046062,0.486758,0.621937,-0.023337,0.584130,0.615383,0.031674
7331,H/WIN_20221122_15_59_43_Pro_00001255.jpg,0.466905,0.603527,-0.005979,0.450994,0.587765,-0.005979,0.433660,0.599534,-0.005979,...,-0.006167,0.578117,0.604436,0.029152,0.450127,0.601484,-0.005979,0.566656,0.607923,0.018814
7332,H/WIN_20221122_15_59_43_Pro_00001256.jpg,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
cat_df = pd.merge(data_table,face_mesh_df.reset_index().rename(columns={"index": "PATH"}))
cat_df.to_csv(data_dir.joinpath("facemesh.csv"), index=None)

In [12]:
import pandas as pd
pd.read_csv("../data/photographed_data/facemesh.csv")

,PATH,HUMAN_ID,CLASS,iris469_x,iris469_y,iris469_z,iris470_x,iris470_y,iris470_z,iris471_x,...,face465_z,face467_x,face467_y,face467_z,face468_x,face468_y,face468_z,face473_x,face473_y,face473_z
0,A/WIN_20221122_15_59_43_Pro_00002521.jpg,A,0,0.050921,0.535985,0.020243,0.041640,0.524391,0.020243,0.034509,...,-0.039480,0.187353,0.491354,-0.062566,0.042635,0.537606,0.020243,0.136263,0.517795,-0.051283
1,A/WIN_20221122_15_59_43_Pro_00002522.jpg,A,0,0.129665,0.522859,0.038498,0.117217,0.512172,0.038498,0.109517,...,-0.014076,0.252512,0.468210,-0.029648,0.119409,0.526774,0.038498,0.213271,0.505276,-0.022296
2,A/WIN_20221122_15_59_43_Pro_00002523.jpg,A,0,0.151061,0.535511,0.053037,0.140435,0.528136,0.053037,0.136560,...,-0.009218,0.263407,0.437156,-0.030716,0.143825,0.540142,0.053037,0.235843,0.473923,-0.020631
3,A/WIN_20221122_15_59_43_Pro_00002524.jpg,A,0,0.223679,0.498827,0.056931,0.213074,0.493173,0.056931,0.205851,...,0.003325,0.334562,0.413146,-0.004158,0.214616,0.504681,0.056931,0.308811,0.437239,0.001437
4,A/WIN_20221122_15_59_43_Pro_00002525.jpg,A,0,0.267782,0.469291,0.055267,0.253962,0.460605,0.055267,0.243765,...,-0.001398,0.374796,0.405864,-0.011049,0.255959,0.474164,0.055267,0.337217,0.431811,-0.004910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7329,H/WIN_20221122_15_59_43_Pro_00001253.jpg,H,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7330,H/WIN_20221122_15_59_43_Pro_00001254.jpg,H,0,0.499172,0.622574,-0.023337,0.487309,0.608572,-0.023337,0.473995,...,0.002184,0.592833,0.623461,0.046062,0.486758,0.621937,-0.023337,0.584130,0.615383,0.031674
7331,H/WIN_20221122_15_59_43_Pro_00001255.jpg,H,0,0.466905,0.603527,-0.005979,0.450994,0.587765,-0.005979,0.433660,...,-0.006167,0.578117,0.604436,0.029152,0.450127,0.601484,-0.005979,0.566656,0.607923,0.018814
7332,H/WIN_20221122_15_59_43_Pro_00001256.jpg,H,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
